In [140]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
from typing import Optional, Union
from pydantic import Field, BaseModel
from langchain_core.messages import BaseMessage, SystemMessage, MessageLikeRepresentation
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.tools import tool

descriptions = {
    "status": """
        This field will be one of the following values:
        - Incomplete or erroneous
            Meaning: Code is incomplete or it is producing errors (it cannot be run succesfully).
        - Incorrect but better
            Meaning: Code is incorrect but better than previous code snippet in terms of syntax, logic and correctness. This means the user is moving in right direction towards the correct code.
        - Incorrect and worse
            Meaning: Code is incorrect and worse than previous code snippet in terms of syntax, logic -  This means the user is moving in wrong direction away from the correct code.
        - Incorrect and same as previous code snippet
            Meaning: Code is incorrect and same as previous code snippet and there is no improvement in the code.
        - Partially correct
            Meaning: Code is partially correct. It covers some of the requirements from the question but not all.
        - Correct but can be improved
            Meaning: Syntax is correct, code does not produce any error on running it and output is as expected BUT the code can be improved (improvements can be in terms of code optimization, better logic, better variable names, better comments, etc.)
        - Correct
            Meaning: Syntax is correct, code does not produce any error on running it and output is as expected. Code is following the guidelines provided in the question. There is no redundancy in the code. Code is optimized and well written.
        """,

    "score": """
        An approximate score out of 100 as the quality of the code snippet provided by user. This score should be based on the quality of code snippet and how well it is following the guidelines provided in the question.
    """,

    "result": """
        - Improving
            Meaning: The given code is better than the previous code snippet in terms of syntax, logic and correctness. This means the user is moving in right direction towards the correct code.
        - Not Improving
            Meaning: The given code is worse than the previous code snippet in terms of syntax, logic or correctness. This means the user is moving in wrong direction away from the correct code.
    """,
}


class Evaluation(BaseModel):
    """Evaluation of the code provided by the user."""

    status: str = Field(default=None, description=descriptions["status"])
    score: str = Field(default=None, description=descriptions["score"])
    result: int = Field(default=0, description=descriptions["result"])

change = 100

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage
from langgraph.graph import START, MessagesState, StateGraph
import time
from langgraph.graph.message import add_messages

model = ChatOpenAI(temperature=0, model="gpt-4o-mini")
model_with_structured_output = model.with_structured_output(Evaluation)

tools = [Evaluation]

workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState) -> MessagesState:
    print("STATE",state)
    try:
        response = model_with_structured_output.invoke(state["messages"])
        print("RESPONSE", response)
    except Exception as e:
        print("EXCEPTION", e)
        print("RESPONSE", response)
        response = {"role":"ai", "content": "I am sorry, I am not able to process this request right now. Please try again later."}
    return {"messages": response}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()

app = workflow.compile(checkpointer=memory)

In [161]:
change += 50
ACTIVE_THREAD_ID = f"test{change}"

config = {"configurable":{"thread_id":ACTIVE_THREAD_ID}}

def get_code(filename:str) -> str:
    with open(filename) as f:
        s = f.read()
        return s

def get_user_code_snippet() -> list[BaseMessage]:
    code_snippet = get_code("test.py")
    return HumanMessage(content=code_snippet)


def get_output(messages: list[BaseMessage]):
    output = app.invoke({"messages": messages}, config)
    output["messages"][-1].pretty_print()

def init():
    message = SystemMessage(content="""
You are a python code quality evaluator.
1. You will receive a question in next prompt.
2. Then you will receive a code snippet by user periodically. 
3. You have to evaluate the quality of code and give the output.
4. Restraint from giving the solution or suggestion or hints.
    """)

    get_output([message])

In [162]:
init()

system_question_template = """Question: {question}"""
change += 1
system_question_prompt = ChatPromptTemplate(["system",system_question_template])
question = "Write a program that takes input first name and last name from the user and passes it to the function greeting_full_name to print the full name of the user with a greeting."
system_question_message = ChatPromptTemplate.from_messages([("system", system_question_template)]).invoke({"question": question}).to_messages()
get_output(system_question_message)

STATE {'messages': [SystemMessage(content='\nYou are a python code quality evaluator.\n1. You will receive a question in next prompt.\n2. Then you will receive a code snippet by user periodically. \n3. You have to evaluate the quality of code and give the output.\n4. Restraint from giving the solution or suggestion or hints.\n    ', additional_kwargs={}, response_metadata={}, id='e5d45740-f7dd-478f-8a8c-53c5d0b52050')]}
EXCEPTION 1 validation error for FinalResponse
final_output
  Field required [type=missing, input_value={'content': 'Please provi...d share your question."}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing


UnboundLocalError: cannot access local variable 'response' where it is not associated with a value

In [ ]:
while True:
    user_message = get_user_code_snippet()
    get_output(user_message)
    time.sleep(5)